In [1]:
import requests
import json
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

"""
Função principal para orquestrar o pipeline de dados.
"""
# --- 1. Inicialização do Spark ---
# Cria uma sessão Spark, que é o ponto de entrada para qualquer funcionalidade do Spark.
# O appName ajuda a identificar sua aplicação na UI do Spark.
spark = SparkSession.builder \
    .appName("ETL_SuicidioXDepressao") \
    .getOrCreate()

print("Sessão Spark iniciada com sucesso.")

# --- 2. Extração (Extract) ---
# Coletamos os dados da API usando a biblioteca 'requests' do Python.
# Esta etapa é executada no Driver do Spark.

URL_SUICIDIO = "https://ghoapi.azureedge.net/api/SDGSUICIDE?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq 'SEX_BTSX' and Dim2 eq 'AGEGROUP_YEARSALL'"
URL_DEPRESSAO = "https://ghoapi.azureedge.net/api/MH_12?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq 'SEX_BTSX'"

# É uma boa prática definir um cabeçalho (header) com uma chave de API.
# Para este exemplo, a API funciona sem chave, mas em produção, você deve usar uma.
headers = {
    "Content-Type": "application/json"
}

def extrair_api(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Lança um erro para respostas com status 4xx/5xx
        dados = response.json().get("value",[])
        print(f"Dados coletados com sucesso! Total de registros retornados: {len(dados)}")
        return dados
        
    except Exception as e:
        print(f"Erro ao chamar a API: {e}")
        return []

dados_suicidio = extrair_api(URL_SUICIDIO)
dados_depressao = extrair_api(URL_DEPRESSAO)

# Se a resposta estiver vazia, não há o que processar.
if not dados_suicidio and not dados_depressao:
    print("Nenhum dado retornado pela API. Encerrando o processo...")
    spark.stop()

# --- 3. Transformação (Transform) ---
# Agora, vamos converter os dados JSON em um DataFrame Spark e tratá-los.

# O Spark pode inferir o schema, mas definir explicitamente é mais seguro e performático.
schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("IndicatorCode", StringType(), True),

    StructField("SpatialDimType", StringType(), True),
    StructField("SpatialDim", StringType(), True),

    StructField("ParentLocationCode", StringType(), True),
    StructField("TimeDimType", StringType(), True),
    StructField("ParentLocation", StringType(), True),

    StructField("Dim1Type", StringType(), True),
    StructField("TimeDim", IntegerType(), True),
    StructField("Dim1", StringType(), True),

    StructField("Dim2Type", StringType(), True),
    StructField("Dim2", StringType(), True),

    StructField("Dim3Type", StringType(), True),
    StructField("Dim3", StringType(), True),

    StructField("DataSourceDimType", StringType(), True),
    StructField("DataSourceDim", StringType(), True),

    StructField("Value", StringType(), True),
    StructField("NumericValue", DoubleType(), True),

    StructField("Low", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Comments", StringType(), True),

    StructField("Date", StringType(), True),
    StructField("TimeDimensionValue", StringType(), True),
    StructField("TimeDimensionBegin", StringType(), True),
    StructField("TimeDimensionEnd", StringType(), True)
])

# Criamos um RDD (Resilient Distributed Dataset) a partir do JSON e depois um DataFrame.
# Isso permite que o Spark distribua o processamento dos dados.
def criar_df_tratado(dados, dataset_nome):
    json_strings = [json.dumps(item) for item in dados]
    rdd = spark.sparkContext.parallelize(json_strings)
    df = spark.read.json(rdd, schema=schema)
    
    # Aplicando transformações para limpar e organizar os dados
    df_tratado = df.select(
        col("SpatialDim").alias("Pais"),
        col("TimeDim").alias("Ano"),
        col("Dim1").alias("Sexo"),
        col("NumericValue").alias("Valor")
    ).withColumn("dataset_origem", lit(dataset_nome))
    
    return df_tratado

df_suicidio = criar_df_tratado(dados_suicidio, "suicidio")
df_depressao = criar_df_tratado(dados_depressao, "depressao")

df_suicidio.printSchema()
df_suicidio.show(5, False)

df_depressao.printSchema()
df_depressao.show(5, False)

# --- 4. Carga (Load) ---
# Salvamos o DataFrame tratado no HDFS em formato Parquet.
# Parquet é um formato colunar otimizado para performance em análises com Spark.

# O caminho no HDFS onde os dados serão salvos.
CAMINHO_SILVER_SUICIDIO = "hdfs://namenode:9000/datalake/silver/suicidio"
CAMINHO_SILVER_DEPRESSAO = "hdfs://namenode:9000/datalake/silver/depressao"

print(f"Salvando dados tratados no HDFS...")

# 'overwrite' substitui os dados se o diretório já existir.
# 'partitionBy' é útil para organizar os dados, mas como já filtramos por ano_mes,
# incluímos ele no caminho para criar uma partição manual.
df_suicidio.write.mode("overwrite").parquet(CAMINHO_SILVER_SUICIDIO)
df_depressao.write.mode("overwrite").parquet(CAMINHO_SILVER_DEPRESSAO)

print("Dados salvos com sucesso no HDFS!")

print("DataFrames Silver carregados:")

df_suicidio.show(5)
df_depressao.show(5)

df_suicidio = df_suicidio.withColumnRenamed("Valor", "Taxa_Suicidio")
df_depressao = df_depressao.withColumnRenamed("Valor", "Taxa_Depressao")

# ======================
# 5. JOIN (Camada GOLD)
# ======================

df_gold = df_suicidio.alias("s") \
    .join(
        df_depressao.alias("d"),
        [
            col("s.Pais") == col("d.Pais"),
            col("s.Ano") == col("d.Ano"),
            col("s.Sexo") == col("d.Sexo")
        ],
        "inner"
    ) \
    .select(
        col("s.Pais"),
        col("s.Ano"),
        col("s.Sexo"),
        col("s.Taxa_Suicidio"),
        col("d.Taxa_Depressao")
    )

print("Resultado do join GOLD:")
df_gold.show(10, truncate=False)

# ======================
# 6. Caminho GOLD
# ======================

CAMINHO_GOLD = "hdfs://namenode:9000/datalake/gold/suicidio_depressao"

# ======================
# 7. Salvando em PARQUET
# ======================

df_gold.write.mode("overwrite").parquet(CAMINHO_GOLD)

print("Arquivo GOLD salvo em Parquet com sucesso!")

# ======================
# 8. (Opcional) Salvar também em CSV – ideal para Power BI
# ======================
url = "jdbc:postgresql://postgres:5432/superset"
tabela = "fato_suicidio_depressao"
props = {
    "user": "superset",
    "password": "superset",
    "driver": "org.postgresql.Driver"
}

df_gold.write \
    .mode("overwrite") \
    .jdbc(url=url, table=tabela, properties=props)

print("Tabela salva com sucesso no PostgreSQL!")

# --- 5. Finalização ---
spark.stop()
print("Sessão Spark finalizada.")

Sessão Spark iniciada com sucesso.
Dados coletados com sucesso! Total de registros retornados: 185
Dados coletados com sucesso! Total de registros retornados: 185
root
 |-- Pais: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Valor: double (nullable = true)
 |-- dataset_origem: string (nullable = false)

+----+----+--------+-----------+--------------+
|Pais|Ano |Sexo    |Valor      |dataset_origem|
+----+----+--------+-----------+--------------+
|GHA |2021|SEX_BTSX|5.316740355|suicidio      |
|JPN |2021|SEX_BTSX|17.43068584|suicidio      |
|TGO |2021|SEX_BTSX|9.340400499|suicidio      |
|LSO |2021|SEX_BTSX|28.66132363|suicidio      |
|AZE |2021|SEX_BTSX|1.558857505|suicidio      |
+----+----+--------+-----------+--------------+
only showing top 5 rows

root
 |-- Pais: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Valor: double (nullable = true)
 |-- dataset_origem: string (nul